# Xây dựng mô hình từ giải thuật SVM trên dữ liệu các con thú trong rừng.

## 1. Khai báo thư viện

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report
import warnings

## 2. Tải và làm sạch dữ liệu

In [3]:
# Tải dữ liệu
df = pd.read_csv("data/data.csv")

# 1. Xử lý dữ liệu thiếu: Xóa các dòng mà cột 'Dangerous' bị trống (NaN)
print(f"Số dòng ban đầu: {len(df)}")
df_clean = df.dropna(subset=['Dangerous'])
print(f"Số dòng sau khi xóa NaN: {len(df_clean)}")

# 2. Mã hóa cột mục tiêu (y): Chuyển 'Yes' -> 1 và 'No' -> 0
df_clean['Dangerous'] = df_clean['Dangerous'].map({'Yes': 1, 'No': 0})

# Tách y (mục tiêu) ra
y = df_clean['Dangerous'].astype(int)

# Tách X (features) là tất cả các cột còn lại
X = df_clean.drop('Dangerous', axis=1)

Số dòng ban đầu: 871
Số dòng sau khi xóa NaN: 869


C:\Users\84794\AppData\Local\Temp\ipykernel_1488\1843973512.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Dangerous'] = df_clean['Dangerous'].map({'Yes': 1, 'No': 0})


## 3. Tiền xử lý - Mã hóa One-Hot

In [4]:
# Áp dụng One-Hot Encoding cho toàn bộ dataframe X
X_encoded = pd.get_dummies(X)

print("\n--- Dữ liệu sau khi mã hóa One-Hot ---")
print(f"Số features ban đầu: {len(X.columns)}")
print(f"Số features sau khi mã hóa: {len(X_encoded.columns)}")
print(X_encoded.head())


--- Dữ liệu sau khi mã hóa One-Hot ---
Số features ban đầu: 6
Số features sau khi mã hóa: 1153
   AnimalName_Birds  AnimalName_Black-tailed deer  AnimalName_Buffaloes  \
0             False                         False                 False   
1             False                         False                 False   
2             False                         False                 False   
3             False                         False                 False   
4             False                         False                 False   

   AnimalName_Cattle  AnimalName_Chicken  AnimalName_Deer  AnimalName_Dog  \
0              False               False            False            True   
1              False               False            False            True   
2              False               False            False            True   
3              False               False            False            True   
4              False               False            False           

## 4. Chia dữ liệu Train / Test

In [ ]:
# Chia dữ liệu (X_encoded và y)
# stratify=y giúp đảm bảo tỷ lệ Yes/No trong tập train và test là tương đương nhau
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42, stratify=y)

## 5. Huấn luyện mô hình SVM

In [6]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
best_svm = None
best_val_acc = -1
best_kernel = None

print("\n--- Bắt đầu huấn luyện SVM ---")

for kernel_name in kernels:
    # Khởi tạo mô hình SVM
    clf = svm.SVC(kernel=kernel_name, probability=True)
    
    # Huấn luyện mô hình trên X_train (đã mã hóa)
    clf.fit(X_train, y_train)
    
    # Đánh giá độ chính xác trên tập X_test
    tmp_val_acc = clf.score(X_test, y_test)
    
    print(f"Kernel: {kernel_name}, Accuracy: {tmp_val_acc * 100:.2f}%")
    
    # Cập nhật mô hình tốt nhất
    if (tmp_val_acc > best_val_acc):
        best_val_acc = tmp_val_acc
        best_svm = clf
        best_kernel = kernel_name

print("\n--- Kết quả ---")
print(f"Best validation accuracy: {best_val_acc * 100:.2f}% với kernel: {best_kernel}")


--- Bắt đầu huấn luyện SVM ---
Kernel: linear, Accuracy: 100.00%
Kernel: poly, Accuracy: 97.70%
Kernel: rbf, Accuracy: 97.70%
Kernel: sigmoid, Accuracy: 98.85%

--- Kết quả ---
Best validation accuracy: 100.00% với kernel: linear


## 6. Đánh giá chi tiết mô hình tốt nhất

In [7]:
# Dùng mô hình tốt nhất để dự đoán trên tập test
y_pred = best_svm.predict(X_test)

# In báo cáo phân loại
print("\n--- Báo cáo phân loại chi tiết (Classification Report) ---")
# target_names=['No' (lớp 0), 'Yes' (lớp 1)]
print(classification_report(y_test, y_pred, target_names=['No', 'Yes']))


--- Báo cáo phân loại chi tiết (Classification Report) ---
              precision    recall  f1-score   support

          No       1.00      1.00      1.00         4
         Yes       1.00      1.00      1.00       170

    accuracy                           1.00       174
   macro avg       1.00      1.00      1.00       174
weighted avg       1.00      1.00      1.00       174

